In [1]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time

warnings.filterwarnings("ignore")

In [2]:
paths = os.listdir(path='F:\\')
bb_path = list(filter(lambda file: file.endswith('BB.csv'), paths))[0]
hh_path = list(filter(lambda file: file.endswith('HH.csv'), paths))[0]
print(bb_path, hh_path)

SVO-AER_2018-01-01--2019-12-31BB.csv SVO-AER_2018-01-01--2019-12-31HH.csv


In [3]:
dates = ['FLTDAT', 'DAIS']
dtypes = {'FLTNUM': 'int16', 'TARIF_RUB': 'float16'}
bb = pd.read_csv('F:\\' + bb_path, delimiter=';', dtype=dtypes, parse_dates=dates)
bb = bb.dropna(how='any', axis=0)
bb.drop_duplicates(inplace=True)
bb['DEPARTURE'] = bb['FLTDAT'] + pd.to_timedelta(bb['FLTTIME'])
bb.head()

,AIRL,CPTK,TDNR,COUN,FLTDAT,FLTTIME,DAIS,MCAR,FLTNUM,ORAP,DSAP,BCLS,FBTD,KOL_CPN,TARIF_RUB,SALE_CURR,TARIF_SALE,USE_STAT,USE_DAT,DEPARTURE
0,555.0,5.190193e+09,5.190193e+09,2.0,2018-01-21,13:35:00,2018-01-20,SU,1124,SVO,AER,E,EVOR,1.0,2740.0,EUR,39.49,USED,2018-01-21,2018-01-21 13:35:00
1,555.0,5.963380e+09,5.963380e+09,4.0,2018-01-10,09:10:00,2017-11-13,SU,1122,SVO,AER,L,LVU,1.0,4744.0,EUR,68.78,USED,2018-01-10,2018-01-10 09:10:00
2,555.0,2.140311e+09,2.140311e+09,1.0,2018-01-11,09:10:00,2017-09-30,SU,1122,SVO,AER,N,NVUR,1.0,1530.0,RUB,1530.0,EXCH,2018-01-04,2018-01-11 09:10:00
3,555.0,2.140303e+09,2.140303e+09,1.0,2018-01-26,17:35:00,2017-09-30,SU,1140,SVO,AER,N,NVUR,1.0,1530.0,RUB,1529.88,USED,2018-01-26,2018-01-26 17:35:00
4,555.0,2.139698e+09,2.139698e+09,1.0,2018-01-06,09:10:00,2017-09-12,SU,1122,SVO,AER,N,NCLRCH25,1.0,1897.0,RUB,1897.11,USED,2018-01-06,2018-01-06 09:10:00


In [25]:
%%time
use_cols = ('SDAT_S', 'FLT_NUM', 'DD', 'SEG_CLASS_CODE', 'FCLCLD',
            'PASS_BK', 'SA', 'AU', 'PASS_DEP', 'NS', 'DTD')
dtypes = {'FLT_NUM': 'float16', 'PASS_BK': 'float16', 'SA': 'float16', 'AU': 'float16',
        'PASS_DEP': 'float16', 'NS': 'float16', 'DTD': 'float16'}
dates = ['SDAT_S', 'DD']
hh = pd.read_csv('F:\\' + hh_path, delimiter=';', dtype=dtypes, parse_dates=dates, usecols=use_cols)
size_gb = f'{sys.getsizeof(hh) / 1024 / 1024 / 1024} Gb'
hh.dropna(how='any', axis=0, inplace=True)
hh.drop_duplicates(inplace=True)
hh[['FLT_NUM', 'PASS_BK', 'SA', 'AU', 'PASS_DEP', 'NS', 'DTD']] = hh[['FLT_NUM', 'PASS_BK', 'SA', 'AU',
                                                           'PASS_DEP', 'NS', 'DTD']].astype(int)
hh.to_csv('F:\\hh_for_profiles.csv', index=False)
print(size_gb)
display(hh.head())

5.1270735105499625 Gb


,SDAT_S,FLT_NUM,DD,SEG_CLASS_CODE,FCLCLD,PASS_BK,SA,AU,PASS_DEP,NS,DTD
0,2018-01-02,1126,2018-01-01,J,1.0,12,0,20,12,0,-1
1,2018-01-02,1126,2018-01-01,C,1.0,0,0,18,0,0,-1
2,2018-01-02,1126,2018-01-01,D,1.0,3,0,16,3,0,-1
3,2018-01-02,1126,2018-01-01,I,1.0,0,0,14,0,0,-1
4,2018-01-02,1126,2018-01-01,Z,1.0,0,0,14,0,0,-1


Wall time: 18min 42s


In [4]:
%%time
dates = ['SDAT_S', 'DD']
dtypes = {'FLT_NUM': 'int16', 'FCLCLD': 'int16', 'PASS_BK': 'int16','SA': 'int16',
          'AU': 'int16', 'PASS_DEP': 'int16', 'NS': 'int16', 'DTD': 'int16'}
hh = pd.read_csv('F:\\hh_for_profiles.csv', dtype=dtypes, parse_dates=dates)

Wall time: 40.3 s


In [31]:
HOURS_SHIFT = pd.Timedelta('1:00:00')
def get_flight(date: datetime, flttime: pd.Timedelta) -> (datetime, pd.DataFrame):
    departure = date + flttime
    print("Выбранная дата/время:", departure, date.strftime("%A"))
    date_minus_52_weeks = date - timedelta(weeks=52)
    date_minus_53_weeks = date_minus_52_weeks - timedelta(weeks=1) + flttime
    date_minus_51_weeks = date_minus_52_weeks + timedelta(weeks=1) + flttime
    print("Даты профиля:")
    print(date_minus_52_weeks.date(), date_minus_53_weeks.date(), date_minus_51_weeks.date(), sep='\n')
    filtered_bb = bb[(date_minus_52_weeks + flttime + HOURS_SHIFT >= bb['DEPARTURE']) &
                    (date_minus_52_weeks + flttime - HOURS_SHIFT <= bb['DEPARTURE']) |
                    (date_minus_53_weeks + HOURS_SHIFT >= bb['DEPARTURE']) &
                    (date_minus_53_weeks - HOURS_SHIFT <= bb['DEPARTURE']) |
                    (date_minus_51_weeks + HOURS_SHIFT >= bb['DEPARTURE']) &
                    (date_minus_51_weeks - HOURS_SHIFT <= bb['DEPARTURE'])]
    print(f"Отфильтрованных данных: {filtered_bb.shape[0]:_}")
    merged = pd.merge(filtered_bb, hh, left_on=['FLTNUM', 'FLTDAT'], right_on=['FLT_NUM', 'DD'])
    merged = merged.dropna(how='any', axis=0)
    merged.drop(columns=['FLTDAT', 'FLTTIME', 'FLT_NUM', 'DD', 'AIRL', 'CPTK', 'TDNR', 'COUN',
                         'DAIS', 'MCAR', 'ORAP', 'DSAP', 'TARIF_SALE', 'SALE_CURR', 'USE_STAT',
                         'FBTD', 'KOL_CPN'],
                inplace=True)
    merged.drop_duplicates(inplace=True)
    print(f"Объединённых данных: {merged.shape[0]:_}")
    return departure, merged

In [32]:
%%time
departure, dynamic_profile = get_flight(datetime.strptime('2019-11-27', '%Y-%m-%d'),
                                        pd.Timedelta('21:20:00'))

Выбранная дата/время: 2019-11-27 21:20:00 Wednesday
Даты профиля:
2018-11-28
2018-11-21
2018-12-05
Отфильтрованных данных: 67
Объединённых данных: 163_812
Wall time: 3.53 s


# Сезонный профиль

## Уникальные рейсы (рейс/дата/время) в сезонном профиле:

In [33]:
unique_flights = (dynamic_profile['FLTNUM'].astype(str) + ' ' + 
                  (dynamic_profile['DEPARTURE']).astype(str)).unique()
print(f'{len(unique_flights)} уникальных рейсов: {unique_flights}')

3 уникальных рейсов: ['1128 2018-11-28 21:20:00' '1128 2018-11-21 21:20:00'
 '1128 2018-12-05 21:20:00']


## Средние цены тарифов:

In [34]:
bcls_mean = dynamic_profile[['BCLS', 'TARIF_RUB']].copy()
bcls_mean = bcls_mean.groupby(by=['BCLS'])['TARIF_RUB'].mean().reset_index()
bcls_mean

,BCLS,TARIF_RUB
0,D,29504.0
1,E,4752.0
2,L,7840.0
3,N,2578.0
4,O,5700.0
5,Q,6632.0
6,R,838.0
7,T,5956.0
8,X,0.0


## Среднее количество и std броней (PASS_BK) в разрезе классов бронирования (SEG_CLASS_CODE) и дней до вылета (DTD):

In [35]:
mean_pass_bk = pd.pivot_table(dynamic_profile, index='DTD', columns='SEG_CLASS_CODE',
                              values='PASS_BK', aggfunc=['mean'])
file_name = f'{departure} +- {str(HOURS_SHIFT).replace("0 days ", "")}.xlsx'.replace(':', '.')
file_name = 'mean_pass_bk s ' + file_name
mean_pass_bk.to_excel(file_name)
mean_pass_bk = pd.read_excel(file_name, index_col=0, skiprows=lambda row: row in [0, 2])
mean_pass_bk = mean_pass_bk.rename_axis("SEG_CLASS_CODE (mean)", axis="columns").rename_axis("DTD")
display(mean_pass_bk)

std_pass_bk = pd.pivot_table(dynamic_profile, index='DTD', columns='SEG_CLASS_CODE',
                             values='PASS_BK', aggfunc=['std'])
file_name = f'{departure} +- {str(HOURS_SHIFT).replace("0 days ", "")}.xlsx'.replace(':', '.')
file_name = 'std_pass_bk s ' + file_name
std_pass_bk.to_excel(file_name)
std_pass_bk = pd.read_excel(file_name, index_col=0, skiprows=lambda row: row in [0, 2])
std_pass_bk = std_pass_bk.rename_axis("SEG_CLASS_CODE (std)", axis="columns").rename_axis("DTD")
display(std_pass_bk)

SEG_CLASS_CODE (mean),B,C,D,E,G,H,I,J,K,L,...,O,P,Q,R,T,U,V,X,Y,Z
DTD,,,,,,,,,,,,,,,,,,,,,
-1,0,0,3.647059,1.941176,0,0.647059,0.235294,0,1.235294,4.117647,...,3.176471,0,30.647059,25.588235,3.058824,0,0,2.588235,0,0
0,0,0,3.647059,1.941176,0,0.647059,0.235294,0,1.235294,4.117647,...,3.176471,0,30.647059,25.588235,3.058824,0,0,2.588235,0,0
1,0,0,3.058824,1.705882,0,0.411765,0.000000,0,0.000000,4.117647,...,2.058824,0,37.823529,27.235294,2.588235,0,0,1.764706,0,0
2,0,0,2.647059,1.705882,0,0.000000,0.000000,0,0.000000,4.117647,...,2.058824,0,37.823529,22.529412,2.588235,0,0,1.764706,0,0
3,0,0,1.470588,1.705882,0,0.000000,0.000000,2,0.000000,2.058824,...,2.058824,0,37.823529,22.294118,2.588235,0,0,1.764706,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,0,0,0.000000,0.000000,0,0.000000,0.000000,2,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0
214,0,0,0.000000,0.000000,0,0.000000,0.000000,2,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0
215,0,0,0.000000,0.000000,0,0.000000,0.000000,2,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0


SEG_CLASS_CODE (std),B,C,D,E,G,H,I,J,K,L,...,O,P,Q,R,T,U,V,X,Y,Z
DTD,,,,,,,,,,,,,,,,,,,,,
-1,0,0,1.554706,1.455214,0,0.485071,0.430562,0,1.498663,4.995542,...,2.152808,0,35.967900,11.949687,2.423886,0,0,1.971273,0,0
0,0,0,1.554706,1.455214,0,0.485071,0.430562,0,1.498663,4.995542,...,2.152808,0,35.967900,11.949687,2.423886,0,0,1.971273,0,0
1,0,0,1.722246,1.337782,0,0.499554,0.000000,0,0.000000,4.995542,...,1.455214,0,45.161307,12.926503,1.940285,0,0,1.538571,0,0
2,0,0,1.554706,1.337782,0,0.000000,0.000000,0,0.000000,4.995542,...,1.455214,0,45.161307,8.128645,1.940285,0,0,1.538571,0,0
3,0,0,1.211943,1.337782,0,0.000000,0.000000,0,0.000000,2.497771,...,1.455214,0,45.161307,7.975453,1.940285,0,0,1.538571,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,0,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0
214,0,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0
215,0,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0


## Среднее количество улетевших пассажиров (PASS_DEP) в разрезе классов (SEG_CLASS_CODE) и дней до вылета (DTD) в момент времени -1:

In [36]:
mean_std_pass_dep = dynamic_profile[dynamic_profile['DTD'] == -1]
mean_std_pass_dep = mean_std_pass_dep.groupby(by=['SEG_CLASS_CODE'])['PASS_DEP'].agg(['mean', 'std'])
file_name = f'{departure} +- {str(HOURS_SHIFT).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_std_pass_dep s ' + file_name
mean_std_pass_dep.to_csv(file_name)
mean_std_pass_dep = pd.read_csv(file_name, index_col=0).reset_index()
display(mean_std_pass_dep)

,SEG_CLASS_CODE,mean,std
0,B,0.000000,0.000000
1,C,0.000000,0.000000
2,D,3.647059,1.554706
3,E,1.941176,1.455214
4,G,0.000000,0.000000
5,H,0.647059,0.485071
6,I,0.235294,0.430562
7,J,0.000000,0.000000
8,K,1.235294,1.498663
9,L,4.117647,4.995542


## Среднее количество неявившихся пассажиров (NS) в разрезе классов (SEG_CLASS_CODE) и дней до вылета (DTD):

In [37]:
mean_ns = dynamic_profile[dynamic_profile['DTD'] == -1]
mean_ns = mean_ns.groupby(by=['SEG_CLASS_CODE'])['NS'].agg(['mean'])
file_name = f'{departure} +- {str(HOURS_SHIFT).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_ns s ' + file_name
mean_ns.to_csv(file_name)
mean_ns = pd.read_csv(file_name, index_col=0).reset_index()
display(mean_ns)

,SEG_CLASS_CODE,mean
0,B,0.000000
1,C,0.000000
2,D,0.000000
3,E,0.000000
4,G,0.000000
5,H,0.000000
6,I,0.000000
7,J,0.000000
8,K,0.000000
9,L,0.000000


# Динамический профиль 

In [7]:
start_date = datetime.strptime('2019-01-01', '%Y-%m-%d')
flight = 1126
filtered_data = hh[(hh['FLT_NUM'] == flight) & (hh['DD'] == start_date)]
days_ago = int(max(filtered_data['DTD'].unique()) + 1)
print("Дней в динамическом профиле", days_ago) # дней между началом продаж и днём вылета
end_date = (start_date - timedelta(days=days_ago)).strftime('%Y-%m-%d')
print("Дата начала продаж:", end_date) # дата начала продаж
filtered_data = hh[(hh['DD'] >= end_date) & (hh['DD'] <= start_date) & (hh['FLT_NUM'] == flight)]
dates_in_dyn_profile = filtered_data['DD'].unique()
print("Количество уникальных дат:", len(dates_in_dyn_profile))
# далее идёт выбор из этих дат

Дней в динамическом профиле 218
Дата начала продаж: 2018-05-28
Количество уникальных дат: 219


In [10]:
%%time
date = datetime.strptime('2018-12-07', '%Y-%m-%d') # input()
flttime = pd.Timedelta('21:20:00') # input()
print(date.date(), str(flttime).replace('0 days ', ''), date.strftime("%A"))
one_day = timedelta(days=1)
date_minus_1_week = pd.to_datetime(date) - pd.DateOffset(weeks=1)
date_minus_2_weeks = pd.to_datetime(date) - pd.DateOffset(weeks=2)
print(f'Выбранная дата: {date.date()}')
print(f'Дата -1 неделя: {date_minus_1_week.date()}')
print(f'Дата -2 недели: {date_minus_2_weeks.date()}')
# 1 неделя назад +- 1 день
date_minus_1_week_gap = [date_minus_1_week,
                         date_minus_1_week + one_day, date_minus_1_week - one_day]
# 2 недели назад +- 1 день
date_minus_2_weeks_gap = [date_minus_2_weeks,
                          date_minus_2_weeks + one_day, date_minus_2_weeks - one_day]
print(date_minus_1_week.date(), date_minus_2_weeks.date(), sep='\n')
# фильтрация по дням
filtered_hh = hh[(hh['DD'].isin(date_minus_1_week_gap)) | (hh['DD'].isin(date_minus_2_weeks_gap))]
print(f"Отфильтрованных данных: {filtered_hh.shape[0]:_}")
merged = pd.merge(bb, filtered_hh, left_on=['FLTNUM', 'FLTDAT'], right_on=['FLT_NUM', 'DD'], how='inner')
merged = merged.dropna(how='any', axis=0)
print(f"Объединённых данных: {merged.shape[0]:_}")
two_hours = pd.Timedelta('2:00:00')
# 1 неделя назад в заданное время +- 2 часа
date_minus_1_week_gap = [date_minus_1_week + flttime + two_hours,
                         date_minus_1_week + flttime - two_hours]
# 2 недели назад в заданное время +- 2 часа
date_minus_2_weeks_gap = [date_minus_2_weeks + flttime + two_hours,
                          date_minus_2_weeks + flttime - two_hours]
# фильтрация по времени +- 2 часа
merged = merged[(date_minus_1_week_gap[0] >= merged['DEPARTURE']) &
                (date_minus_1_week_gap[1] <= merged['DEPARTURE']) |
                (date_minus_2_weeks_gap[0] >= merged['DEPARTURE']) &
                (date_minus_2_weeks_gap[1] <= merged['DEPARTURE'])]
merged = merged.drop(columns=['DD', 'FLT_NUM'], axis=1)
merged = merged.dropna(how='any')
print(f"Отфильрованных по времени данных: {merged.shape[0]:_}")

2018-12-07 21:20:00 Friday
Выбранная дата: 2018-12-07
Дата -1 неделя: 2018-11-30
Дата -2 недели: 2018-11-23
2018-11-30
2018-11-23
Отфильтрованных данных: 567_703
Объединённых данных: 19_964_440
Отфильрованных по времени данных: 1_586_310
Wall time: 12.5 s


## Уникальные рейсы (рейс/дата/время) в сезонном профиле:

In [11]:
unique_flights = (merged['FLTNUM'].astype(str) + ' ' +
                 (merged['FLTDAT'] + merged['FLTTIME']).astype(str)).unique()
print(f'{len(unique_flights)} уникальных рейсов: {unique_flights}')

4 уникальных рейсов: ['1128 2018-11-30 21:20:00' '1130 2018-11-23 23:00:00'
 '1130 2018-11-30 23:00:00' '1128 2018-11-23 21:20:00']


## Средние цены тарифов для классов в сезонном профиле:

In [12]:
bcls_mean = merged[['BCLS', 'TARIF_RUB']].copy()
bcls_mean = bcls_mean.groupby(by=['BCLS'])['TARIF_RUB'].mean().reset_index()
bcls_mean

,BCLS,TARIF_RUB
0,C,34464.0
1,D,29504.0
2,E,3062.0
3,M,0.0
4,N,1905.0
5,O,0.0
6,Q,5732.0
7,R,751.0
8,T,4652.0


## Среднее количество броней (PASS_BK) в разрезе классов (BCLS) и дней до вылета (DTD):

In [13]:
mean_pass_bk = pd.pivot_table(merged, index='DTD', columns='BCLS', values='PASS_BK',
                              aggfunc=['mean'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_pass_bk d ' + file_name
mean_pass_bk.to_csv(file_name)
mean_pass_bk = pd.read_csv(file_name, index_col=0, skiprows=lambda row: row in [0, 2])
mean_pass_bk = mean_pass_bk.rename_axis('DTD').rename_axis('BCLS', axis=1)
display(mean_pass_bk)

std_pass_bk = pd.pivot_table(merged, index='DTD', columns='BCLS', values='PASS_BK',
                             aggfunc=['std'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'std_pass_bk d ' + file_name
std_pass_bk.to_csv(file_name)
std_pass_bk = pd.read_csv(file_name, index_col=0, skiprows=lambda row: row in [0, 2])
std_pass_bk = std_pass_bk.rename_axis('DTD').rename_axis('BCLS', axis=1)
display(std_pass_bk)

BCLS,C,D,E,M,N,O,Q,R,T
DTD,,,,,,,,,
-1,5.227273,5.227273,7.272727,7.272727,6.339246,7.272727,5.227273,5.902893,5.227273
0,5.227273,5.227273,7.272727,7.272727,6.339246,7.272727,5.227273,5.902893,5.227273
1,5.454545,5.454545,7.045455,7.045455,5.717849,7.045455,5.454545,4.979339,5.454545
2,5.454545,5.454545,6.454545,6.454545,5.253880,6.454545,5.454545,4.603306,5.454545
3,5.363636,5.363636,6.227273,6.227273,4.896896,6.227273,5.363636,4.134298,5.363636
...,...,...,...,...,...,...,...,...,...
213,0.181818,0.181818,0.136364,0.136364,0.124723,0.136364,0.181818,0.115702,0.181818
214,0.136364,0.136364,0.136364,0.136364,0.116962,0.136364,0.136364,0.105372,0.136364
215,0.136364,0.136364,0.136364,0.136364,0.116962,0.136364,0.136364,0.105372,0.136364


BCLS,C,D,E,M,N,O,Q,R,T
DTD,,,,,,,,,
-1,10.188602,10.129878,15.338675,15.492479,14.222040,15.403185,10.100894,14.022027,10.129878
0,10.129878,10.100894,15.327032,15.403185,14.220069,15.359113,10.086495,14.018403,10.100894
1,10.826648,10.795670,14.799040,14.872569,13.153956,14.830015,10.780281,12.086834,10.795670
2,10.852098,10.821047,14.735073,14.808284,12.761134,14.765914,10.805622,11.569526,10.821047
3,10.770524,10.739707,14.636512,14.709233,11.957480,14.667147,10.724397,10.131974,10.739707
...,...,...,...,...,...,...,...,...,...
213,0.492366,0.490957,0.457159,0.459430,0.447361,0.458116,0.490257,0.439687,0.490957
214,0.459430,0.458116,0.457159,0.459430,0.440732,0.458116,0.457463,0.430434,0.458116
215,0.459430,0.458116,0.457159,0.459430,0.440732,0.458116,0.457463,0.430434,0.458116


## Среднее количество улетевших пассажиров (PASS_DEP) в разрезе классов (BCLS) и дней до вылета (DTD) в момент времени -1:

In [14]:
mean_std_pass_dep = merged[merged['DTD'] == -1]
mean_std_pass_dep = mean_std_pass_dep.groupby(by=['BCLS'])['PASS_DEP'].agg(['mean', 'std'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_std_pass_dep d ' + file_name
mean_std_pass_dep.to_csv(file_name)
mean_std_pass_dep = pd.read_csv(file_name, index_col=0)
display(mean_std_pass_dep)

,mean,std
BCLS,,
C,5.227273,10.188602
D,5.227273,10.129878
E,7.272727,15.338675
M,7.272727,15.492479
N,6.223947,13.837972
O,7.272727,15.403185
Q,5.227273,10.100894
R,5.704545,13.382508
T,5.227273,10.129878


## Среднее количество неявившихся пассажиров (NS) в разрезе классов (BCLS) и дней до вылета (DTD):

In [15]:
mean_ns = merged[merged['DTD'] == -1]
mean_ns = mean_ns.groupby(by=['BCLS'])['NS'].agg(['mean'])
file_name = f'{str(date + flttime)} +- {str(two_hours).replace("0 days ", "")}.csv'.replace(':', '.')
file_name = 'mean_ns d ' + file_name
mean_ns.to_csv(file_name)
mean_ns = pd.read_csv(file_name, index_col=0)
display(mean_ns)

,mean
BCLS,
C,0.272727
D,0.272727
E,0.045455
M,0.045455
N,0.093126
O,0.045455
Q,0.272727
R,0.121901
T,0.272727
